In [122]:
import requests
import re 
from tqdm import tqdm
from bs4 import BeautifulSoup
import json
url = "https://nips.cc/Conferences/2022/Schedule?showEvent="


In [3]:
response = requests.get(url)
soup = BeautifulSoup(response.content,"html5lib")

In [56]:
all_show_details  = []
for data in soup.findAll("div",onclick=True):
    if "showDetail" in  data.get("onclick"):
        show_details = re.findall(r"\AshowDetail\([0-9]+\)",data.get("onclick"))
        all_show_details.extend(show_details)
len(all_show_details),len(set(all_show_details))

(3239, 3239)

In [64]:
event_ids = []
for show_details in all_show_details:
    res = re.findall("[0-9]+",show_details)
    event_ids.extend([int(x) for x in res])
len(event_ids),len(set(event_ids))

(3239, 3239)

In [116]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from tqdm import tqdm 

def concurrency(function,data,length,method="thread"):
    if method=="thread":
        print("Mode :",method)
        with ThreadPoolExecutor(5) as pe:
           res = list(tqdm(pe.map(function,data),total=length))
           return res
    elif method=="process":
        print("Mode :",method)
        with ProcessPoolExecutor(5) as pe:
           res = list(tqdm(pe.map(function,data),total=length))
           return res
    else:
        print("ERROR")
        raise

In [117]:
base_event_url = "https://nips.cc/Conferences/2022/Schedule?showEvent="

def fetch_data_from_event_id(event_id):
    data = {"event_id":event_id}
    url_ = base_event_url+str(event_id)
    event_res = requests.get(url_)
    event_soup = BeautifulSoup(event_res.content,"html5lib")
    paper_title = event_soup.findAll("div",class_="maincardBody")
    abstract = event_soup.findAll("div",class_= "abstractContainer")
    data["title"] = paper_title[0].text
    data["abstract"] = abstract[0].text
    data["url"] = url_
    return data

In [118]:
results = concurrency(fetch_data_from_event_id,event_ids,len(event_ids))

Mode : thread


100%|██████████| 3239/3239 [04:04<00:00, 13.24it/s] 


In [123]:
# all_data = results.copy()

In [ ]:
import random
random.shuffle(results)

In [124]:
results_dict = {x["event_id"]:x for x in results}

In [126]:
json.dump(results_dict,open("./scraped_data/neurips2022.json","w"))